In [1]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.models import load_model

import numpy as np
import scipy
import scipy.sparse as sp

import matplotlib.pyplot as plt

import random, os, h5py, math, time, glob

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F


Using TensorFlow backend.


In [2]:

class CNNClassifier(nn.Module) :
    
    def __init__(self) :
        super(CNNClassifier, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=(0, 0))
        self.act1 = nn.ReLU()
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=(0, 0))
        self.act2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d((2, 2))
        self.drop2 = nn.Dropout(p=0.25)
        
        self.fc3 = nn.Linear(in_features=9216, out_features=128)
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(p=0.5)
        
        self.fc4 = nn.Linear(in_features=128, out_features=10)
        self.act4 = nn.Softmax(dim=1)
        
        self.use_cuda = True if torch.cuda.is_available() else False
        
    def forward(self, x):
        
        x = self.act1(self.conv1(x))
        x = self.act2(self.conv2(x))
        
        x = self.drop2(self.maxpool2(x))
        
        x = x.transpose(1, 3)
        x = x.reshape(-1, 9216)
        x = self.drop3(self.act3(self.fc3(x)))
        
        y = self.act4(self.fc4(x))
        
        return y


In [32]:
#Load pytorch model skeleton

model_pytorch = CNNClassifier()


In [3]:
#Load keras model

model_keras = load_model("saved_models/mnist_binarized_cnn_10_digits.h5")
model_keras.trainable = False

model_keras.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.Adadelta(),
    metrics=['accuracy']
)


In [34]:
#Collect weights from keras model

conv_1_weight, conv_1_bias = model_keras.get_layer('conv2d_1').get_weights()

conv_2_weight, conv_2_bias = model_keras.get_layer('conv2d_2').get_weights()

dense_1_weight, dense_1_bias = model_keras.get_layer('dense_1').get_weights()

dense_2_weight, dense_2_bias = model_keras.get_layer('dense_2').get_weights()



In [35]:
#Print keras model weight shapes

print(conv_1_weight.shape)
print(conv_1_bias.shape)
print("----------")
print(conv_2_weight.shape)
print(conv_2_bias.shape)
print("----------")
print(dense_1_weight.shape)
print(dense_1_bias.shape)
print("----------")
print(dense_2_weight.shape)
print(dense_2_bias.shape)


(3, 3, 1, 32)
(32,)
----------
(3, 3, 32, 64)
(64,)
----------
(9216, 128)
(128,)
----------
(128, 10)
(10,)


In [36]:
#Print pytorch model weight shapes


print(model_pytorch.conv1.weight.shape)
print(model_pytorch.conv1.bias.shape)
print("----------")
print(model_pytorch.conv2.weight.shape)
print(model_pytorch.conv2.bias.shape)
print("----------")
print(model_pytorch.fc3.weight.shape)
print(model_pytorch.fc3.bias.shape)
print("----------")
print(model_pytorch.fc4.weight.shape)
print(model_pytorch.fc4.bias.shape)



torch.Size([32, 1, 3, 3])
torch.Size([32])
----------
torch.Size([64, 32, 3, 3])
torch.Size([64])
----------
torch.Size([128, 9216])
torch.Size([128])
----------
torch.Size([10, 128])
torch.Size([10])


In [37]:
#Manually transfer model weights from keras to pytorch

with torch.no_grad() :
    
    model_pytorch.conv1.weight = nn.Parameter(torch.FloatTensor(np.transpose(conv_1_weight, (3, 2, 1, 0))))
    model_pytorch.conv1.bias = nn.Parameter(torch.FloatTensor(conv_1_bias))
    
    model_pytorch.conv2.weight = nn.Parameter(torch.FloatTensor(np.transpose(conv_2_weight, (3, 2, 1, 0))))
    model_pytorch.conv2.bias = nn.Parameter(torch.FloatTensor(conv_2_bias))
    
    model_pytorch.fc3.weight = nn.Parameter(torch.FloatTensor(np.transpose(dense_1_weight, (1, 0))))
    model_pytorch.fc3.bias = nn.Parameter(torch.FloatTensor(dense_1_bias))
    
    model_pytorch.fc4.weight = nn.Parameter(torch.FloatTensor(np.transpose(dense_2_weight, (1, 0))))
    model_pytorch.fc4.bias = nn.Parameter(torch.FloatTensor(dense_2_bias))
    

In [38]:
#Save pytorch model

torch.save(model_pytorch.state_dict(), "saved_models/mnist_binarized_cnn_10_digits_pytorch.pth")


In [4]:
#Load MNIST data

dataset_name = "mnist_3_vs_5"

img_rows, img_cols = 28, 28

num_classes = 10
batch_size = 32

included_classes = { 3, 5 }

(x_train, y_train), (x_test, y_test) = mnist.load_data()

keep_index_train = []
for i in range(y_train.shape[0]) :
    if y_train[i] in included_classes :
        keep_index_train.append(i)

keep_index_test = []
for i in range(y_test.shape[0]) :
    if y_test[i] in included_classes :
        keep_index_test.append(i)

x_train = x_train[keep_index_train]
x_test = x_test[keep_index_test]
y_train = y_train[keep_index_train]
y_test = y_test[keep_index_test]

n_train = int((x_train.shape[0] // batch_size) * batch_size)
n_test = int((x_test.shape[0] // batch_size) * batch_size)
x_train = x_train[:n_train]
x_test = x_test[:n_test]
y_train = y_train[:n_train]
y_test = y_test[:n_test]


x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print("x_train.shape = " + str(x_train.shape))

print("n train samples = " + str(x_train.shape[0]))
print("n test samples = " + str(x_test.shape[0]))

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

#Binarize images

def _binarize_images(x, val_thresh=0.5) :
    
    x_bin = np.zeros(x.shape)
    x_bin[x >= val_thresh] = 1.
    
    return x_bin

x_train = _binarize_images(x_train, val_thresh=0.5)
x_test = _binarize_images(x_test, val_thresh=0.5)


x_train.shape = (11552, 28, 28, 1)
n train samples = 11552
n test samples = 1888


In [5]:
#Load predictor model

model_pytorch = CNNClassifier()
_ = model_pytorch.load_state_dict(torch.load("saved_models/mnist_binarized_cnn_10_digits_pytorch.pth"))


In [8]:
#Predict using keras model
y_pred_keras = model_keras.predict(x=[x_test[:32]], batch_size=1)

#Predict using pytorch model
model_pytorch.eval()
        
input_var = Variable(torch.FloatTensor(np.transpose(x_test[:32], (0, 3, 2, 1))))
input_var = input_var.cuda() if model_pytorch.use_cuda else input_var

y_pred_pytorch = model_pytorch(input_var).data.cpu().numpy()


In [9]:

for i, [p_keras, p_pytorch] in enumerate(zip(y_pred_keras.tolist(), y_pred_pytorch.tolist())) :
    print("--------------------")
    print("Sequence " + str(i))
    print("prob (keras)   = " + str(np.round(p_keras, 4)))
    print("prob (pytorch) = " + str(np.round(p_pytorch, 4)))


--------------------
Sequence 0
prob (keras)   = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
prob (pytorch) = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
--------------------
Sequence 1
prob (keras)   = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
prob (pytorch) = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
--------------------
Sequence 2
prob (keras)   = [0.000e+00 0.000e+00 0.000e+00 9.984e-01 0.000e+00 2.000e-04 0.000e+00
 0.000e+00 1.300e-03 0.000e+00]
prob (pytorch) = [0.000e+00 0.000e+00 0.000e+00 9.984e-01 0.000e+00 2.000e-04 0.000e+00
 0.000e+00 1.300e-03 0.000e+00]
--------------------
Sequence 3
prob (keras)   = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
prob (pytorch) = [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
--------------------
Sequence 4
prob (keras)   = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
prob (pytorch) = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
--------------------
Sequence 5
prob (keras)   = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
prob (pytorch) = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
--------------------
Sequence 6
prob (keras)   = [0. 0. 0. 1. 0. 0. 0. 0. 0. 0